In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

## 1. Installation

In [3]:
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 281.4/281.4 MB 4.1 MB/s eta 0:00:0000:0100:01
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.0/199.0 KB 12.5 MB/s eta 0:00:00
  Created wheel for pyspark: filename=pyspark-3.2.1-py2.py3-none-any.whl size=281853642 sha256=26133dec15059019b3179a93f385674d9733c53b513fe32b78fe4ec39ff00174
  Stored in directory: /root/.cache/pip/wheels/9f/f5/07/7cd8017084dce4e93e84e92efd1e1d5334db05f2e83bcef74f
Successfully built pyspark
  Attempting uninstall: py4j
    Found existing installation: py4j 0.10.9.5
    Uninstalling py4j-0.10.9.5:
      Successfully uninstalled py4j-0.10.9.5


## 2. Spark Session

In [4]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.getOrCreate()

Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/04/21 10:06:59 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


## 3. Dummy data creation

In [5]:
sampleData = [
    {
        "employee": "James", "department": "Sales", "state": "NY", "salary": 90000, "age": 34, "bonus": 10000
    },
    {
        "employee": "Michael", "department": "Sales", "state": "NY", "salary": 86000, "age": 56, "bonus": 20000
    },
    {
        "employee": "Robert", "department": "Sales", "state": "CA", "salary": 81000, "age": 39, "bonus": 23000
    },
    {
        "employee": "Maria", "department": "Finance", "state": "CA", "salary": 90000, "age": 24, "bonus": 23000
    },
    {
        "employee": "Raman", "department": "Finance", "state": "CA", "salary": 99000, "age": 40, "bonus": 24000
    },
    {
        "employee": "Scott", "department": "Finance", "state": "NY", "salary": 83000, "age": 36, "bonus": 19000
    },
    {
        "employee": "Jen", "department": "Finance", "state": "NY", "salary": 79000, "age": 53, "bonus": 15000
    },
    {
        "employee": "Jeff", "department": "Marketing", "state": "CA", "salary": 80000, "age": 25, "bonus": 18000
    },
    {
        "employee": "Kumar", "department": "Marketing", "state": "NY", "salary": 91000, "age": 50, "bonus": 21000
    }
]


## 4. create Spark DataFrame and show

In [6]:
# to read from a .csv file
# df = spark.read.csv("<file_path>")

df = spark.createDataFrame(sampleData)
df.show()

+---+-----+----------+--------+------+-----+
|age|bonus|department|employee|salary|state|
+---+-----+----------+--------+------+-----+
| 34|10000|     Sales|   James| 90000|   NY|
| 56|20000|     Sales| Michael| 86000|   NY|
| 39|23000|     Sales|  Robert| 81000|   CA|
| 24|23000|   Finance|   Maria| 90000|   CA|
| 40|24000|   Finance|   Raman| 99000|   CA|
| 36|19000|   Finance|   Scott| 83000|   NY|
| 53|15000|   Finance|     Jen| 79000|   NY|
| 25|18000| Marketing|    Jeff| 80000|   CA|
| 50|21000| Marketing|   Kumar| 91000|   NY|
+---+-----+----------+--------+------+-----+



### show as a pandas df

In [7]:
df.limit(5).toPandas()

,age,bonus,department,employee,salary,state
0,34,10000,Sales,James,90000,NY
1,56,20000,Sales,Michael,86000,NY
2,39,23000,Sales,Robert,81000,CA
3,24,23000,Finance,Maria,90000,CA
4,40,24000,Finance,Raman,99000,CA


### print schema

In [8]:
df.printSchema()

root
 |-- age: long (nullable = true)
 |-- bonus: long (nullable = true)
 |-- department: string (nullable = true)
 |-- employee: string (nullable = true)
 |-- salary: long (nullable = true)
 |-- state: string (nullable = true)



### rename columns

In [9]:
df = df.withColumnRenamed("employee", "employee_name")
df.printSchema()

## rename for all columns
# df = df.toDF(*["age_xy", "bonus_xy", "department_xy", "employee_xy", "salary_xy", "state_xy"])

root
 |-- age: long (nullable = true)
 |-- bonus: long (nullable = true)
 |-- department: string (nullable = true)
 |-- employee_name: string (nullable = true)
 |-- salary: long (nullable = true)
 |-- state: string (nullable = true)



## select columns

In [10]:
df1 = df.select("employee_name", "age", "state")
df1.show()

+-------------+---+-----+
|employee_name|age|state|
+-------------+---+-----+
|        James| 34|   NY|
|      Michael| 56|   NY|
|       Robert| 39|   CA|
|        Maria| 24|   CA|
|        Raman| 40|   CA|
|        Scott| 36|   NY|
|          Jen| 53|   NY|
|         Jeff| 25|   CA|
|        Kumar| 50|   NY|
+-------------+---+-----+



## sort

In [11]:
## by default, in ascending order
df.sort('age').show()

+---+-----+----------+-------------+------+-----+
|age|bonus|department|employee_name|salary|state|
+---+-----+----------+-------------+------+-----+
| 24|23000|   Finance|        Maria| 90000|   CA|
| 25|18000| Marketing|         Jeff| 80000|   CA|
| 34|10000|     Sales|        James| 90000|   NY|
| 36|19000|   Finance|        Scott| 83000|   NY|
| 39|23000|     Sales|       Robert| 81000|   CA|
| 40|24000|   Finance|        Raman| 99000|   CA|
| 50|21000| Marketing|        Kumar| 91000|   NY|
| 53|15000|   Finance|          Jen| 79000|   NY|
| 56|20000|     Sales|      Michael| 86000|   NY|
+---+-----+----------+-------------+------+-----+



In [12]:
## sort in descending order
from pyspark.sql import functions as F

df.sort(F.desc('age')).show()

+---+-----+----------+-------------+------+-----+
|age|bonus|department|employee_name|salary|state|
+---+-----+----------+-------------+------+-----+
| 56|20000|     Sales|      Michael| 86000|   NY|
| 53|15000|   Finance|          Jen| 79000|   NY|
| 50|21000| Marketing|        Kumar| 91000|   NY|
| 40|24000|   Finance|        Raman| 99000|   CA|
| 39|23000|     Sales|       Robert| 81000|   CA|
| 36|19000|   Finance|        Scott| 83000|   NY|
| 34|10000|     Sales|        James| 90000|   NY|
| 25|18000| Marketing|         Jeff| 80000|   CA|
| 24|23000|   Finance|        Maria| 90000|   CA|
+---+-----+----------+-------------+------+-----+



## create a new column using Spark UDF

In [13]:
# normal python function
def salary_in_k(x):
    return x/1000

# convert the above function to spark UDF and return type of function (Typecast)
from pyspark.sql import types as T

salary_in_k_udf = F.udf(salary_in_k, T.DoubleType())

# creatinfg column
df = df.withColumn('salary_in_k', salary_in_k_udf(F.col('salary')))

df.show()
                   

+---+-----+----------+-------------+------+-----+-----------+
|age|bonus|department|employee_name|salary|state|salary_in_k|
+---+-----+----------+-------------+------+-----+-----------+
| 34|10000|     Sales|        James| 90000|   NY|       90.0|
| 56|20000|     Sales|      Michael| 86000|   NY|       86.0|
| 39|23000|     Sales|       Robert| 81000|   CA|       81.0|
| 24|23000|   Finance|        Maria| 90000|   CA|       90.0|
| 40|24000|   Finance|        Raman| 99000|   CA|       99.0|
| 36|19000|   Finance|        Scott| 83000|   NY|       83.0|
| 53|15000|   Finance|          Jen| 79000|   NY|       79.0|
| 25|18000| Marketing|         Jeff| 80000|   CA|       80.0|
| 50|21000| Marketing|        Kumar| 91000|   NY|       91.0|
+---+-----+----------+-------------+------+-----+-----------+



## Filter

In [14]:
df.filter((df.age >= 50) & (df.salary_in_k >= 80.0)).show()

+---+-----+----------+-------------+------+-----+-----------+
|age|bonus|department|employee_name|salary|state|salary_in_k|
+---+-----+----------+-------------+------+-----+-----------+
| 56|20000|     Sales|      Michael| 86000|   NY|       86.0|
| 50|21000| Marketing|        Kumar| 91000|   NY|       91.0|
+---+-----+----------+-------------+------+-----+-----------+



## groupBy

In [15]:
# groupBy on single column with sum agg
df.groupBy('department').sum('salary').show()

+----------+-----------+
|department|sum(salary)|
+----------+-----------+
|     Sales|     257000|
|   Finance|     351000|
| Marketing|     171000|
+----------+-----------+



In [16]:
# groupBy on multiple columns
df.groupBy('department', 'state').sum('salary', 'bonus').show()

+----------+-----+-----------+----------+
|department|state|sum(salary)|sum(bonus)|
+----------+-----+-----------+----------+
|     Sales|   NY|     176000|     30000|
|     Sales|   CA|      81000|     23000|
|   Finance|   CA|     189000|     47000|
|   Finance|   NY|     162000|     34000|
| Marketing|   NY|      91000|     21000|
| Marketing|   CA|      80000|     18000|
+----------+-----+-----------+----------+



In [17]:
# running more aggregations at a time
df.groupBy('department').agg(F.sum('salary'), F.avg('salary'),
                            F.min('bonus'), F.max('bonus')).show()

+----------+-----------+-----------------+----------+----------+
|department|sum(salary)|      avg(salary)|min(bonus)|max(bonus)|
+----------+-----------+-----------------+----------+----------+
|     Sales|     257000|85666.66666666667|     10000|     23000|
|   Finance|     351000|          87750.0|     15000|     24000|
| Marketing|     171000|          85500.0|     18000|     21000|
+----------+-----------+-----------------+----------+----------+



In [18]:
# using filter and alias on the aggreated data
df.groupBy('department').agg(F.sum('salary').alias('sum_salary'), 
                             F.avg('salary').alias('avg_salary'),
                             F.min('bonus').alias('min_bonus'), 
                             F.max('bonus').alias('max_bonus')).where(
                                                                F.col('min_bonus') >= 15000).show()

+----------+----------+----------+---------+---------+
|department|sum_salary|avg_salary|min_bonus|max_bonus|
+----------+----------+----------+---------+---------+
|   Finance|    351000|   87750.0|    15000|    24000|
| Marketing|    171000|   85500.0|    18000|    21000|
+----------+----------+----------+---------+---------+



In [19]:
## collect aggregated list
df.groupBy('department').agg(F.collect_list('state')).alias('state_list').show()

+----------+-------------------+
|department|collect_list(state)|
+----------+-------------------+
|     Sales|       [NY, NY, CA]|
|   Finance|   [CA, CA, NY, NY]|
| Marketing|           [CA, NY]|
+----------+-------------------+



## Joins

In [20]:
df1 = df.select("employee_name", "age", "state", "salary_in_k")
df2 = df.select("salary", "department", "bonus", "employee_name", "age")
df1.show(), df2.show()

+-------------+---+-----+-----------+
|employee_name|age|state|salary_in_k|
+-------------+---+-----+-----------+
|        James| 34|   NY|       90.0|
|      Michael| 56|   NY|       86.0|
|       Robert| 39|   CA|       81.0|
|        Maria| 24|   CA|       90.0|
|        Raman| 40|   CA|       99.0|
|        Scott| 36|   NY|       83.0|
|          Jen| 53|   NY|       79.0|
|         Jeff| 25|   CA|       80.0|
|        Kumar| 50|   NY|       91.0|
+-------------+---+-----+-----------+

+------+----------+-----+-------------+---+
|salary|department|bonus|employee_name|age|
+------+----------+-----+-------------+---+
| 90000|     Sales|10000|        James| 34|
| 86000|     Sales|20000|      Michael| 56|
| 81000|     Sales|23000|       Robert| 39|
| 90000|   Finance|23000|        Maria| 24|
| 99000|   Finance|24000|        Raman| 40|
| 83000|   Finance|19000|        Scott| 36|
| 79000|   Finance|15000|          Jen| 53|
| 80000| Marketing|18000|         Jeff| 25|
| 91000| Marketing|21

(None, None)

In [21]:
df = df1.join(df2, ["employee_name", "age"], how='inner') # how='left'/'right'
df.show()

+-------------+---+-----+-----------+------+----------+-----+
|employee_name|age|state|salary_in_k|salary|department|bonus|
+-------------+---+-----+-----------+------+----------+-----+
|        James| 34|   NY|       90.0| 90000|     Sales|10000|
|         Jeff| 25|   CA|       80.0| 80000| Marketing|18000|
|          Jen| 53|   NY|       79.0| 79000|   Finance|15000|
|        Kumar| 50|   NY|       91.0| 91000| Marketing|21000|
|        Maria| 24|   CA|       90.0| 90000|   Finance|23000|
|      Michael| 56|   NY|       86.0| 86000|     Sales|20000|
|        Raman| 40|   CA|       99.0| 99000|   Finance|24000|
|       Robert| 39|   CA|       81.0| 81000|     Sales|23000|
|        Scott| 36|   NY|       83.0| 83000|   Finance|19000|
+-------------+---+-----+-----------+------+----------+-----+



In [22]:
# when one table is too big and other is too small -> try broadcasting/map side joins
df = df1.join(F.broadcast(df2), ["employee_name", "age"], how='inner')
df.show()

+-------------+---+-----+-----------+------+----------+-----+
|employee_name|age|state|salary_in_k|salary|department|bonus|
+-------------+---+-----+-----------+------+----------+-----+
|        James| 34|   NY|       90.0| 90000|     Sales|10000|
|      Michael| 56|   NY|       86.0| 86000|     Sales|20000|
|       Robert| 39|   CA|       81.0| 81000|     Sales|23000|
|        Maria| 24|   CA|       90.0| 90000|   Finance|23000|
|        Raman| 40|   CA|       99.0| 99000|   Finance|24000|
|        Scott| 36|   NY|       83.0| 83000|   Finance|19000|
|          Jen| 53|   NY|       79.0| 79000|   Finance|15000|
|         Jeff| 25|   CA|       80.0| 80000| Marketing|18000|
|        Kumar| 50|   NY|       91.0| 91000| Marketing|21000|
+-------------+---+-----+-----------+------+----------+-----+



## Use SQL with DataFrame

In [26]:
# register the df to a table
df.registerTempTable('df_table')

from pyspark.sql import SQLContext
sqlContext = SQLContext(spark)

sqlContext.sql('select * from df_table where age >= 50').show()


/opt/conda/lib/python3.7/site-packages/pyspark/sql/context.py:79: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  FutureWarning


+-------------+---+-----+-----------+------+----------+-----+
|employee_name|age|state|salary_in_k|salary|department|bonus|
+-------------+---+-----+-----------+------+----------+-----+
|      Michael| 56|   NY|       86.0| 86000|     Sales|20000|
|          Jen| 53|   NY|       79.0| 79000|   Finance|15000|
|        Kumar| 50|   NY|       91.0| 91000| Marketing|21000|
+-------------+---+-----+-----------+------+----------+-----+



## Using RDD

In [28]:
import math
from pyspark.sql import Row

def row_function(row):
    # convert row to python dict()
    row_dict = row.asDict()
    
    # add a new key and value
    row_dict['exp_bonus'] = float(np.log(row_dict['bonus']))
    
    # convert dict back to row 
    newRow = Row(**row_dict)
    
    return newRow

# convert df to RDD
df_rdd = df.rdd

# apply above function to RDD
df_rdd_new = df_rdd.map(lambda row: row_function(row))

# convert rdd back to DataFrame
df_new = spark.createDataFrame(df_rdd_new)

df_new.show()

+-------------+---+-----+-----------+------+----------+-----+------------------+
|employee_name|age|state|salary_in_k|salary|department|bonus|         exp_bonus|
+-------------+---+-----+-----------+------+----------+-----+------------------+
|        James| 34|   NY|       90.0| 90000|     Sales|10000| 9.210340371976184|
|      Michael| 56|   NY|       86.0| 86000|     Sales|20000| 9.903487552536127|
|       Robert| 39|   CA|       81.0| 81000|     Sales|23000|10.043249494911286|
|        Maria| 24|   CA|       90.0| 90000|   Finance|23000|10.043249494911286|
|        Raman| 40|   CA|       99.0| 99000|   Finance|24000|10.085809109330082|
|        Scott| 36|   NY|       83.0| 83000|   Finance|19000| 9.852194258148577|
|          Jen| 53|   NY|       79.0| 79000|   Finance|15000| 9.615805480084347|
|         Jeff| 25|   CA|       80.0| 80000| Marketing|18000| 9.798127036878302|
|        Kumar| 50|   NY|       91.0| 91000| Marketing|21000|  9.95227771670556|
+-------------+---+-----+---

## Using Pandas UDF

In [29]:
df.printSchema()

root
 |-- employee_name: string (nullable = true)
 |-- age: long (nullable = true)
 |-- state: string (nullable = true)
 |-- salary_in_k: double (nullable = true)
 |-- salary: long (nullable = true)
 |-- department: string (nullable = true)
 |-- bonus: long (nullable = true)



In [33]:
## get normalized bonus grouped by department

# Declare the schema for the output of our function
outSchema = T.StructType([T.StructField('employee_name', T.StringType(),True),
                        T.StructField('age', T.LongType(),True),
                        T.StructField('state', T.StringType(),True),
                        T.StructField('salary', T.LongType(),True),
                        T.StructField('salary_in_k', T.DoubleType(),True),
                        T.StructField('department', T.StringType(),True),
                        T.StructField('bonus', T.LongType(),True),
                        T.StructField('normalized_bonus', T.DoubleType(),True)
                       ])

# decorate our function with pandas_udf decorator
@F.pandas_udf(outSchema, F.PandasUDFType.GROUPED_MAP)
def subtract_mean(pdf):
    # pdf is a pandas.DataFrame
    v = pdf.bonus
    v = v - v.mean()
    pdf['normalized_bonus'] = v
    
    return pdf

confirmed_groupwise_normalization = df.groupby("department").apply(subtract_mean)

confirmed_groupwise_normalization.toPandas()

,employee_name,age,state,salary,salary_in_k,department,bonus,normalized_bonus
0,Maria,24,CA,90000,90.0,Finance,23000,2750.000000
1,Raman,40,CA,99000,99.0,Finance,24000,3750.000000
2,Scott,36,NY,83000,83.0,Finance,19000,-1250.000000
3,Jen,53,NY,79000,79.0,Finance,15000,-5250.000000
4,Jeff,25,CA,80000,80.0,Marketing,18000,-1500.000000
5,Kumar,50,NY,91000,91.0,Marketing,21000,1500.000000
6,James,34,NY,90000,90.0,Sales,10000,-7666.666667
7,Michael,56,NY,86000,86.0,Sales,20000,2333.333333
8,Robert,39,CA,81000,81.0,Sales,23000,5333.333333


## Mllib

In [34]:
from pyspark.ml import Pipeline
from pyspark.ml.feature import VectorAssembler, StringIndexer

# category encoding
indexers = [StringIndexer(inputCol=column, outputCol=column+'_idx'
                         ) for column in ["department", "employee_name", "state"]]

# create pieline for indexers
pipeline = Pipeline(stages=indexers)

# fit indexer pipeline on df and transform
indexed = pipeline.fit(df).transform(df)
indexed.show()

+-------------+---+-----+-----------+------+----------+-----+--------------+-----------------+---------+
|employee_name|age|state|salary_in_k|salary|department|bonus|department_idx|employee_name_idx|state_idx|
+-------------+---+-----+-----------+------+----------+-----+--------------+-----------------+---------+
|        James| 34|   NY|       90.0| 90000|     Sales|10000|           1.0|              0.0|      0.0|
|      Michael| 56|   NY|       86.0| 86000|     Sales|20000|           1.0|              5.0|      0.0|
|       Robert| 39|   CA|       81.0| 81000|     Sales|23000|           1.0|              7.0|      1.0|
|        Maria| 24|   CA|       90.0| 90000|   Finance|23000|           0.0|              4.0|      1.0|
|        Raman| 40|   CA|       99.0| 99000|   Finance|24000|           0.0|              6.0|      1.0|
|        Scott| 36|   NY|       83.0| 83000|   Finance|19000|           0.0|              8.0|      0.0|
|          Jen| 53|   NY|       79.0| 79000|   Finance|

In [36]:
## correlation matrix
from pyspark.ml.stat import Correlation

# convert to vector column first
vector_col = 'corr_features'
corr_cols = list(set(indexed.columns) - set(["department", "employee_name", "state"]))

# vector assembler
assembler = VectorAssembler(inputCols=corr_cols, outputCol=vector_col)
indexed_vector = assembler.transform(indexed).select(vector_col)

# get correlation matrix
matrix = Correlation.corr(indexed_vector, vector_col)

result = matrix.collect()[0]['pearson({})'.format(vector_col)].values
result

/opt/conda/lib/python3.7/site-packages/pyspark/sql/context.py:127: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  FutureWarning


array([ 1.        ,  1.        ,  0.29052843,  0.1369899 ,  0.06978632,
       -0.15799063, -0.03379182,  1.        ,  1.        ,  0.29052843,
        0.1369899 ,  0.06978632, -0.15799063, -0.03379182,  0.29052843,
        0.29052843,  1.        ,  0.59007577,  0.69498803, -0.11942294,
       -0.03488578,  0.1369899 ,  0.1369899 ,  0.59007577,  1.        ,
        0.17320508, -0.03162278, -0.63245553,  0.06978632,  0.06978632,
        0.69498803,  0.17320508,  1.        , -0.43817805,  0.11907012,
       -0.15799063, -0.15799063, -0.11942294, -0.03162278, -0.43817805,
        1.        ,  0.0173913 , -0.03379182, -0.03379182, -0.03488578,
       -0.63245553,  0.11907012,  0.0173913 ,  1.        ])

In [37]:
# corr matrix as a DataFrame
pd.DataFrame(result.reshape(-1, len(corr_cols)), columns=corr_cols, index=corr_cols)

,salary_in_k,salary,bonus,state_idx,employee_name_idx,department_idx,age
salary_in_k,1.000000,1.000000,0.290528,0.136990,0.069786,-0.157991,-0.033792
salary,1.000000,1.000000,0.290528,0.136990,0.069786,-0.157991,-0.033792
bonus,0.290528,0.290528,1.000000,0.590076,0.694988,-0.119423,-0.034886
state_idx,0.136990,0.136990,0.590076,1.000000,0.173205,-0.031623,-0.632456
employee_name_idx,0.069786,0.069786,0.694988,0.173205,1.000000,-0.438178,0.119070
department_idx,-0.157991,-0.157991,-0.119423,-0.031623,-0.438178,1.000000,0.017391
age,-0.033792,-0.033792,-0.034886,-0.632456,0.119070,0.017391,1.000000
